In [ ]:
#%pip install azure-storage-blob


from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# Define connection parameters
connection_string = "DefaultEndpointsProtocol=https;AccountName=pipldata;AccountKey=lg9Q9j97R0C7M4CFKwUaj+VTXe6uHCYTNdrb289x97ZK2DoNom/4y9HR85t+AStTBNEMQ==;Endpointsuf]sfix=core.windows.net"
container_name = "data1"
blob_name = "googleplaystore.csv"

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the blob client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download blob content
blob_data = blob_client.download_blob().readall()

# Load data into a DataFrame
data = pd.read_csv(io.BytesIO(blob_data))


In [ ]:
%pip list | grep azure-storage-blob


azure-storage-blob                12.22.0


In [ ]:
import pkg_resources

installed_packages = pkg_resources.working_set
installed_package_names = {package.project_name for package in installed_packages}

print("azure-storage-blob" in installed_package_names)


True


In [ ]:
from azure.storage.blob import BlobServiceClient

print("azure-storage-blob module imported successfully.")


azure-storage-blob module imported successfully.


In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# Define connection parameters
connection_string = "DefaultEndpointsProtocol=https;AccountName=pipowderdata;AccountKey=HQ64OxOUbl7R0C7M4CFKwUaj+5hg70SHCYTNdrb289x97ZK2DoNom/4y9HR85t+AStTBNEMQ==;EndpointSuffix=core.windows.net"
container_name = "data"
blob_name = "googleplaystore.csv"

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the blob client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download blob content
blob_data = blob_client.download_blob().readall()

# Load data into a DataFrame
data = pd.read_csv(io.BytesIO(blob_data))


In [ ]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Playstore").getOrCreate()

In [ ]:
df = spark.createDataFrame(data)
df.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [ ]:
df.count()


Out[48]: 10841

In [ ]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [ ]:
from pyspark.sql.window import Window
df_filtered = df.filter(col("Reviews").isNotNull() & col("Current Ver").isNotNull())

# Define a window partitioned by 'App' and ordered by 'Reviews' descending
windowSpec = Window.partitionBy("App", "Current Ver").orderBy(col("Reviews").desc())

# Add a row number column to identify the top entry per 'App' and 'Current Ver'
df_with_row_num = df_filtered.withColumn("row_number", row_number().over(windowSpec))

# Filter to keep only the top entry per 'App' and 'Current Ver'
df1 = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

In [ ]:
df1.count()

Out[52]: 9678

In [ ]:
df1 = df1.drop('Current Ver', 'Android Ver', 'Size', 'Content Rating', 'Last Updated')

In [ ]:
df1.show()

+--------------------+-------------------+------+-------+-----------+----+-----+-----------------+
|                 App|           Category|Rating|Reviews|   Installs|Type|Price|           Genres|
+--------------------+-------------------+------+-------+-----------+----+-----+-----------------+
|"i DT" Fútbol. To...|             SPORTS|  null|     27|       500+|Free|    0|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1,000,000+|Free|    0|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|    10,000+|Free|    0|           Comics|
|                  .R|              TOOLS|   4.5|    259|    10,000+|Free|    0|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|    10,000+|Free|    0|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|       100+|Free|    0| News & Magazines|
|      1. FC Köln App|             SPORTS|   4.6|   2019|   100,000+|Free|    0|           Sports|
|10 Best F

In [ ]:
df1.printSchema()


root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df1 = df1.withColumn("Reviews", df["Reviews"].cast(IntegerType()))\
    .withColumn("Installs", regexp_replace("Installs", "[^0-9]", "").cast(IntegerType()))\
    .withColumn("Price", regexp_replace("Price", "[$]", "").cast(IntegerType()))

In [ ]:
df1.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [ ]:
df1 = df1.withColumn(
    "Type",
    when(col("Type").isNull() | (col("Type") == 0), "Free").otherwise(col("Type"))
)

In [ ]:
df1.show()

+--------------------+-------------------+------+-------+--------+----+-----+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+-----------------+
|"i DT" Fútbol. To...|             SPORTS|  null|     27|     500|Free|    0|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|    0|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|    0|           Comics|
|                  .R|              TOOLS|   4.5|    259|   10000|Free|    0|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|   10000|Free|    0|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|     100|Free|    0| News & Magazines|
|      1. FC Köln App|             SPORTS|   4.6|   2019|  100000|Free|    0|           Sports|
|10 Best Foods for...| HEALTH_AND_FITNES

In [ ]:
df1.createOrReplaceTempView("apps")

In [ ]:
spark.sql("select * from apps").show()

+--------------------+-------------------+------+-------+--------+----+-----+-----------------+
|                 App|           Category|Rating|Reviews|Installs|Type|Price|           Genres|
+--------------------+-------------------+------+-------+--------+----+-----+-----------------+
|"i DT" Fútbol. To...|             SPORTS|  null|     27|     500|Free|    0|           Sports|
|+Download 4 Insta...|             SOCIAL|   4.5|  40467| 1000000|Free|    0|           Social|
|- Free Comics - C...|             COMICS|   3.5|    115|   10000|Free|    0|           Comics|
|                  .R|              TOOLS|   4.5|    259|   10000|Free|    0|            Tools|
|              /u/app|      COMMUNICATION|   4.7|    573|   10000|Free|    0|    Communication|
|              058.ba| NEWS_AND_MAGAZINES|   4.4|     27|     100|Free|    0| News & Magazines|
|      1. FC Köln App|             SPORTS|   4.6|   2019|  100000|Free|    0|           Sports|
|10 Best Foods for...| HEALTH_AND_FITNES

In [ ]:

from pyspark.sql.functions import *

# Azure SQL COnnection
jdbcHostname = "datenew.database.windows.net"
jdbcDatabase = "playdata"
jdbcPort = 1432
username = "justin"
password = "just@22042"
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.PostgreSQLServerDriver"
}




In [ ]:
# Write DataFrame to Azure SQL Database
df1.write \
  .mode("overwrite") \
  .jdbc(url=jdbcUrl, table="data3", properties=connectionProperties)

 Top 10 apps with most reviews

In [ ]:
%sql
SELECT App, Reviews
FROM apps
ORDER BY Reviews DESC
LIMIT 10;



App,Reviews
Facebook,78158306
WhatsApp Messenger,69119316
Instagram,66577446
Messenger – Text and Video Chat for Free,56646578
Clash of Clans,44893888
Clean Master- Space Cleaner & Antivirus,42916526
Subway Surfers,27725352
YouTube,25655305
"Security Master - Antivirus, VPN, AppLock, Booster",24900999
Clash Royale,23136735


Databricks visualization. Run in Databricks to view.

Top 10 installed apps 

In [ ]:
%sql
SELECT App, Installs
FROM apps
ORDER BY Installs DESC
LIMIT 10;


App,Installs
Google Chrome: Fast & Secure,1000000000
Hangouts,1000000000
Google Drive,1000000000
Facebook,1000000000
Google News,1000000000
Google Play Games,1000000000
Google+,1000000000
Google Play Movies & TV,1000000000
Gmail,1000000000
Google Street View,1000000000


In [ ]:
%sql
SELECT Type, COUNT(App) AS App_Count
FROM apps
GROUP BY Type;


Type,App_Count
Free,8924
Paid,754


Databricks visualization. Run in Databricks to view.

Category-wise Distribution of Apps


In [ ]:
%sql
SELECT Category, COUNT(*) AS Number_of_Apps
FROM apps
GROUP BY Category
ORDER BY Number_of_Apps DESC;


Category,Number_of_Apps
FAMILY,1875
GAME,950
TOOLS,828
BUSINESS,420
MEDICAL,400
PERSONALIZATION,376
PRODUCTIVITY,375
LIFESTYLE,369
FINANCE,345
SPORTS,325


Databricks visualization. Run in Databricks to view.

Top Paid Apps

In [ ]:
%sql
SELECT App, Price
FROM apps
WHERE Type = 'Paid'
ORDER BY Price DESC
LIMIT 10;


App,Price
I'm Rich - Trump Edition,400
I Am Rich Pro,399
I am Rich,399
I am Rich Plus,399
I Am Rich Premium,399
I am Rich!,399
I AM RICH PRO PLUS,399
I am rich(premium),399
I am rich (Most expensive app),399
I am rich,399


Top Paid Apps with Maximum Ratings

In [ ]:
%sql
SELECT App, Rating
FROM apps
WHERE Type = 'Paid'
ORDER BY Rating DESC
LIMIT 10;



App,Rating
AP Art History Flashcards,5.0
CA Speakers,5.0
CI 174 Gray Icon Pack,5.0
AJ Gray Dark Icon Pack,5.0
Android P Style Icon Pack,5.0
AC DC Power Monitor,5.0
BC MVA Fines,5.0
30WPM Amateur ham radio Koch CW Morse code trainer,5.0
AJ Blue Icon Pack,5.0
ADS-B Driver,5.0
